In [24]:
# Import packages

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.cross_validation import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from patsy import dmatrices

In [5]:
dta = pd.read_csv('../data/dayData.csv')

In [6]:
dta.describe()

,Unnamed: 0,household_key,DAY,QUANTITY,BASE_SPEND_AMT,NET_SPEND_AMT,LOY_CARD_DISC,COUPON_DISC,GET_EGGS,WEEK
count,25797.000000,25797.000000,25797.000000,25797.000000,25797.000000,25797.000000,25797.000000,25797.000000,25797.000000,25797.0
mean,12899.000000,505.790402,101.607086,14.598752,42.941644,36.737954,-6.060541,-0.143149,0.146063,0.0
std,7447.096783,292.033469,58.799244,17.332974,51.432689,43.380989,8.682502,0.921875,0.367696,0.0
min,1.000000,1.000000,0.000000,1.000000,0.210000,0.210000,-208.020000,-43.280000,0.000000,0.0
25%,6450.000000,257.000000,51.000000,3.000000,10.060000,8.970000,-7.830000,0.000000,0.000000,0.0
50%,12899.000000,511.000000,102.000000,8.000000,24.190000,21.010000,-2.970000,0.000000,0.000000,0.0
75%,19348.000000,761.000000,152.000000,19.000000,55.730000,47.560000,-0.790000,0.000000,0.000000,0.0
max,25797.000000,1000.000000,204.000000,261.000000,816.790000,681.650000,0.000000,0.000000,3.000000,0.0


In [7]:
dta = dta.drop('Unnamed: 0', 1)

In [8]:
# Create X and y matrices
# X is the input feature matrix
# y is the column vector of the parameter we want to predict

y, X = dmatrices('GET_EGGS ~ DAY + QUANTITY + BASE_SPEND_AMT + NET_SPEND_AMT + LOY_CARD_DISC + COUPON_DISC', dta, return_type="dataframe")

In [36]:
# Percentage of data with eggs

print(y['GET_EGGS'].value_counts())
freq = y.sum(axis=0)[0] / y.shape[0]
print(freq)

0.0    22160
1.0     3510
2.0      123
3.0        4
Name: GET_EGGS, dtype: int64
0.146063495755


In [37]:
# Baseline predictions 
# Guessing 0 for everything

baselinepredict = [0 for x in range(0, y_test.shape[0])]

In [39]:
# Briar score on the xvaldiation

metrics.brier_score_loss(y_test, baselinepredict)

0.00025839793281653745

In [ ]:
# Cross Validation folds with 20/80 split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [41]:
# Logistic regression model

model = LogisticRegression()
model.fit(X_train, y_train)
logisticpredict = model.predict(X_test)

/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [44]:
# How many 1's and 0's of the prediction

print(np.count_nonzero(logisticpredict==1))
print(np.count_nonzero(logisticpredict==0))

267
7473


In [45]:
# Briar score on the xvalidation

metrics.brier_score_loss(y_test, logisticpredict)

0.034496124031007755

In [46]:
# Random forest model

model2 = RandomForestClassifier()
model2.fit(X_train, y_train)
forestpredict = model2.predict(X_test)

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


In [47]:
# How many 1's and 0's of the prediction

print(np.count_nonzero(forestpredict==1))
print(np.count_nonzero(forestpredict==0))

326
7414


In [48]:
# Briar score on the xvalidation

metrics.brier_score_loss(y_test, forestpredict)

0.042118863049095609

In [49]:
import xgboost as xgb

In [50]:
# XGB Boost model

gbm = xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05).fit(X_train, y_train)
xgbpredict = gbm.predict(X_test)

/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:108: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
# How many 1's and 0's of the prediction

print(np.count_nonzero(forestpredict==1))
print(np.count_nonzero(forestpredict==0))

In [32]:
metrics.brier_score_loss(y_test, predictions)

0.01925064599483204